### Packages

In [0]:
import statsmodels
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.tsa.ar_model import AR
import math
from math import sqrt
from math import isnan
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from google.colab import drive 
drive.mount('/content/gdrive', force_remount =True)

Mounted at /content/gdrive


### Week 3 - MA and ARMA Models

### 1. MA Model Specification

Model where you regress a variable against its current error and past error term. MA(1) model is:

$y_t = \phi_0 + u_t + \theta_1 u_{t-1}$

Again error term normally distributed with zero mean, constant variance and zero autocovariance.

$u_t = N(0,\sigma^2_u)$

MA(q) model is:

$y_t = \phi_0 + u_t + \theta_1 u_{t-1} + ... \theta_q u_{t-q}$

Note: p is lags for AR terms, q is lags for MA terms.

### 2. Benefits of MA Models - Turning into its AR Form (Given Stationarity Assumption)

MA model can generate an infinite autogressive lag structure. Meaning if we wanted to generate a large-p AR(p) mode, we can do so even when the sample size is relatively small. It would be difficult to estimate the AR model but easy to estimate the MA model.

Using lag operators, MA(1) model is:

$y_t = \phi_0 + u_t + \theta_1 u_{t-1}$

$y_t = \phi_0 + (1+\theta_1L) u_t$

We multiply both sides by the inverse of $(1+\theta_1L)$:

$(1+\theta_1L)^{-1} y_t = \frac{\phi_0}{(1+\theta_1)} +  u_t$

Question: why does the L do away?

Given the assumption of stationarity ($|\theta_1|<1$), this series follows a geometeric decaying progression that converges.

$(1+\theta_1L)^{-1} = 1 - \theta_1 L + \theta_1^2L^2 - ... + ...$

This means the above result is:
$(1 - \theta_1 L + \theta_1^2L^2 - ... + ...)y_t = \frac{\phi_0}{(1+\theta_1)} +  u_t$

Without lag operators:
$y_t = \frac{\phi_0}{(1+\theta_1)} + \theta_1 y_{t-1} - \theta^2_1 y_{t-2} + ... - ... + u_t$

Here, given stationarity, a MA(1) model is the same as a restricted AR($\infty$) model, where the AR parameters are restricted as:

$\phi_i = (-)^{i + 1} \theta_1^i$






### 3. MA(1) Model Properties - Unconditional Mean

MA(1) model is:

$y_t = \phi_0 + u_t + \theta_1 u_{t-1}$

1. Take unconditional mean:

$E(y_t) = \phi_0 + E(u_t) + \theta_1 E(u_{t-1})$

2. Property of error term having zero mean:

$\mu = E(y_t) = \phi_0$

### 4. MA(1) Model Properties - Unconditional Variance

MA(1) model is:

$y_t = \phi_0 + u_t + \theta_1 u_{t-1}$

1. Mean form:

$\mu = E(y_t) = \phi_0$

2. Subtract MA(1) model from mean form:

$y_t - \mu = u_t + \theta_1 u_{t-1}$

3. Square both sides:

$(y_t - \mu)^2 = u^2_t + \theta^2_1 u^2_{t-1} + 2\theta_1 u_t u_{t-1}$

4. Take unconditional expectation:

$\gamma_0 = E((y_t - \mu)^2) = E(u^2_t) + \theta^2_1 E(u^2_{t-1}) + 2\theta_1 E(u_t u_{t-1})$

5. Remember $u_t$ is independent of $u_{t-1}$:

$\gamma_0 = E((y_t - \mu)^2) = E(u^2_t) + \theta^2_1 E(u^2_{t-1})$

6. Apply homoskedastic error variance property:

$\sigma^2_u = E(u^2_t) = E(u^2_{t-1})$

$\gamma_0 = E((y_t - \mu)^2) = \sigma^2_u + \theta^2_1 \sigma^2_u = (1 + \theta^2_1)\sigma^2_u$




### 5. MA(1) Model Properties - First Order Autocovariance

First Order Autocovariance defined as:

$\gamma_1 = E((y_t - \mu)(y_{t-1} - \mu))$

1. Consider mean deviation form:

$y_t - \mu = u_t + \theta_1 u_{t-1}$

2. Substitute in equation:

$\gamma_1 = E((u_t + \theta_1 u_{t-1})(u_{t-1} + \theta_1 u_{t-2}))$

3. Expand:

$\gamma_1 = E(u_tu_{t-1} + \theta_1 u_tu_{t-2} + \theta_1 u^2_{t-1} + \theta^2_1u_{t-1}u_{t-2})$

4. Apply fact that $u_t$ is independent to past lags:

$\gamma_1 = \theta_1 E(u^2_{t-1}) = \theta_1 \sigma^2_u$


### 6. Second Order and $p$th Order Autocovariance (Check for MA(1) Model!)

Second Order Autocovariance defined as:

$\gamma_2 = E((y_t - \mu)(y_{t-2} - \mu))$

1. Consider mean deviation form:

$y_t - \mu = u_t + \theta_1 u_{t-1}$

2. Substitute in equation:

$\gamma_2 = E((u_t + \theta_1 u_{t-1})(u_{t-2} + \theta_1 u_{t-3}))$

3. Expand:

$\gamma_2 = E(u_tu_{t-2} + \theta_1 u_tu_{t-3} + \theta_1 u_{t-1}u_{t-2} + \theta^2_1u_{t-1}u_{t-3})$

4. Apply fact that $u_t$ is independent to past lags:

$\gamma_2 = 0$

$p$th Order Autocovariance:

$\gamma_p = E((y_t - \mu)(y_{t-p} - \mu))$

For the MA(1) model:

$\gamma_p = 0$


### 7. First Order and $p$th Order Autocorrelation for MA(1) Model

First Order Autocorrelation is:

$\rho_1 = \frac{\gamma_1}{\gamma_0} = \frac{\theta_1 \sigma^2_u}{(1 + \theta^2_1)\sigma^2_u} = \frac{\theta_1 }{(1 + \theta^2_1)}$

Second Order and $p$th Order Autocovariance is 0.

This implies show that the ACF for a MA(1) has a spike at lag 1.

### 8. MA Model Estimation

Assume error term is distributed as $N(0,\sigma^2_u)$, the MA parameters requires to be estimated are:

$\Theta = {\phi_0, \theta_1, \theta_2, ..., \theta_q, \sigma^2_u}$

We estimate these using maximum likelihood, due to the moving average component.

We aim to choose parameters that maximise the following log likelihood function:

$\log L = -\frac{1}{2}\log 2\pi - -\frac{1}{2}\log\sigma^2_u - \frac{1}{2\sigma^2_uT} \sum_{t=1}^{T} u^2_t$

Where $u_t = y_t - \phi_0 - \theta_1u_{t-1} - \theta_2u_{t-2} ...-\theta_q u_{t-q}$

**Example: Unemployment Rate**

In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')
import numpy as np
import pandas as pd
from pandas import datetime
def parser(x):
    return datetime.strptime(x, '%Y-%m-%d')
series = pd.read_csv('gdrive/My Drive/Colab Notebooks/Time Series/macro.csv', parse_dates = [0], date_parser = parser)
pd.options.display.float_format = '{:.2f}'.format
series.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


FileNotFoundError: ignored

In [0]:
Y = series['URATE']
from statsmodels.tsa.arima_model import ARMA
model = ARMA(Y, order=(0, 1))
model_fit = model.fit(disp=False,method='mle')
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                  URATE   No. Observations:                  199
Model:                     ARMA(0, 1)   Log Likelihood                -306.881
Method:                           mle   S.D. of innovations              1.116
Date:                Fri, 04 Oct 2019   AIC                            619.762
Time:                        06:40:08   BIC                            629.642
Sample:                             0   HQIC                           623.761
                                                                              
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           6.2083      0.158     39.332      0.000       5.899       6.518
ma.L1.URATE     1.0000      0.018     56.353      0.000       0.965       1.035
                                    Roots       

In [0]:
Y = series['URATE']
model = ARMA(Y, order=(0, 2))
model_fit = model.fit(disp=False,method='mle')
print(model_fit.summary())

ValueError: ignored

###9. MA Model Forecasting - Weakness with MA Models



We want to know $y_{t+1}$.

MA model tells us: $y_{t+1} = \phi_0 + u_{t+1} + \theta_1u_t$

We have fitted values for $\phi_0$ and $\theta_1u_t$ and our expectation of $u_{t+1}$ is 0:

$\hat y_{t+1} = \hat \phi_0 + \hat \theta_1u_t$

Note for 2-step ahead and further forecasts, we only get $\hat \phi_0$:

$\hat y_{t+2} = \hat \phi_0 + \hat \theta_1u_{t+1} = \hat \phi_0$

This is simply a property of the MA(1) model.

In general, for a MA(q) model, the forecasts for q + 1 periods ahead and onwards are all the same. This property commonly referred to having finite memory.

In contrast, AR models have infinite memory.

### 10. ARMA Models

ARMA(p,q) model can be used to allow for both AR and MA dynamics:

$y_t = \phi_0 + \phi_1y_{t-1} + \phi_2y_{t-2} + ... + \phi_p y_{t-p} + u_t + \theta_1 u_{t-1} + \theta_2 u_{t-2} + ... + \theta_q u_{t - q}$

Written more compactly using lag operators:

$y_t = \phi_1 y_{t-1} - ... - \phi_p y_{t-p} = \phi_0 + u_t + \theta_1 u_{t-1} + ... + \theta_q u_{t-q}$

$(1=\phi_1 L - ... - \phi_p L^p)y_t = (1 + \theta_1 L + ... + \theta_q L^q)u_t$

### 11. Special Case ARMA(1,1) 


Remember MA(1) process can be shown to have infinite AR representation given stationarity. ARMA process can be the same, basically MA(1) given $\phi_1 = 0$.

### 12. ARMA Model Estimation

Same as MA process estimation, use maximum likelihood estimation.

We estimate the ARMA model perameters:

Assume error term is distributed as $N(0,\sigma^2_u)$, the MA parameters requires to be estimated are:

$\Theta = {\phi_0, \theta_1, \theta_2, ..., \theta_q, \sigma^2_u, \phi_1, \phi_2, ..., \phi_p}$

We aim to choose parameters that maximise the following log likelihood function (same as MA L-L function):

$\log L = -\frac{1}{2}\log 2\pi - -\frac{1}{2}\log\sigma^2_u - \frac{1}{2\sigma^2_uT} \sum_{t=1}^{T} u^2_t$

Where $u_t = y_t - \phi_0 - \theta_1u_{t-1} - \theta_2u_{t-2} ...-\theta_q u_{t-q} - \phi_1 y_{t-1} - ... - \phi_p y_{t-p}$

In [0]:
Y = series['URATE']
from statsmodels.tsa.arima_model import ARMA
model = ARMA(Y, order=(1, 1))
model_fit = model.fit(disp=False,method='mle')
print(model_fit.summary())

NameError: ignored

### 13. ARMA Model Forecasting

Same as before.

ARMA models have long memory due to the AR component of the model.

With ex-post forecasts, we can compare model performance by looking at the RMSE (lower is better).

### 14. Pooling Forecasts

Instead of choosing between models and selecting the best one based on lowest RMSE.

You can combine and pool the models.

You have various model forecasts of $\hat y_t$ and you  combine them by assigning a weight to each model $\hat y_t$ and the weights add to 1.

You can choose weights equally/unweighted, weighted by MSE, or by OLS regression.

By pooling forecasts from an AR and MA model, this effectively generates forecasts for an ARMA model.

Has no thereotical justification, only statistical performance benefits. 

### 15. ARMAX Model

Simply adds an explanatory variable to the ARMA model.

Estimation down by nonlinear least squares.

Forecasting requires some estimate or forecast or scenario analysis of your exogenous variables as well.

Otherwise, just do a multivariate time series and model all variables together.

### Week 4 - Multivariate VAR Models

###1. VAR Specification

We have N variables $[y_{1t},y_{2t}, ..., y_{Nt}]$

Model is specified as:

$y_t = \Phi_0 + \Phi_1 y_{t-1} + \Phi_2 y_{t-2} + ... + \Phi_p y_{t-p} + u_t$

Basically we regress each variable by all variable lags. Hence, we have $N\times 1$ constants $\Phi_0$ and $N\times p$ parameters and $N\times 1$ error terms.

The error terms are non-autocorrelated with zero mean and covariance, same as before - in vector notation, this is:


$\Omega = E(u_t u_t')$




###2. VAR Estimation

Estimation by doing $N$ number of OLS estimations for each $N$ variable, regressing on all variables for $p$ lags and a constant.

OLS estimator for VAR properties:
1.   Asymptotically efficient (i.e. produces smallest standard errors of any other estimator), because all explanatory variables are the same

MLE estimator for VAR proeprties:
1.   More useful if the set of explanatory variables in the VAR are not necessarily the same in each equation
2.   More useful if VAR is extended to SVAR to allow for more structural information
3. The df-adjusted log-likelihood is used to determine the overall lag length $p$ of the VAR

MLE estimation involves:
*   Again assuming $u_t$ is normally distributed with 0 mean and covariance matrix $\Omega$
*   Want to estimate parameters $\theta = [\Phi_0, \Phi_1, ..., \Phi_p, \Omega]$
*   Find parameters that maximise the conditional/average log-likelihood function: $\frac{\log L(\theta)}{T-p}-\frac{N}{2} - \frac{1}{2}\log |\Omega|-\frac{1}{2(T-p)}\sum^T_{t=p+1}u_t'\Omega^{-1}u_t$
*   Hence, the MLE estimates for our parameters are $\hat \theta_{MLE} = \arg \min_\theta \frac{\log  L(\theta)}{T-p}$
*   Equivalence Result: estimator equivalent to OLS estimator
*   The MLE estimator of the error covariance matrix terms are $\frac{1}{T}\sum^T_{t=1}\hat u_{it} \hat u_{jt}$
*   Log likelihood can be intepreted as the overall residual sum of squares for the system in a multivariate setting












###3. VAR Lag Testing

Use AIC, SC, HQ, measures that find the the highest R-squared or log-likelihood while penalising additional lags.

$AIC = -2\frac{\log L(\hat \theta)}{T-p} + 2\frac {K}{T-p}$

$SC = -2\frac{\log L(\hat \theta)}{T-p} + 2\frac {\log (T-p) K}{T-p}$

$HQ = -2\frac{\log L(\hat \theta)}{T-p} + 2\frac {\log(\log(T-p))K}{T-p}$

Where $K$ tells us the number of estimated parameters. Optimal lag structure minimises these statistics.

Including redundant variables could give more information, but issues can result in inefficient parameter estimates (i.e. higher standard errors), less precise forecasts (i.e. higher RMSE).

Information statistic properties:
*   SC and HQ are consistent tests (choose correct lag structure as $T -> \infty$)
*   In small samples, SC tends to be too conservative and choose too short of a lag length
*   Hence, HQ is generally the best choice



###4. VAR Forecasting

One advantage of ex-post forecasting is it proivides an alternative lag-testing approach. i.e. use RMSE vs AIC, SIC, HQ.

### Week 5 - VAR and Multivariate Causality Tests

### 1. Causality Analysis Motivation

VARs can contain many estimated parameters.

We want to know which variables are important or not.

You can do Granger causality tests, where you do joint tests on whetehr lags of other variabels help to predict $y$ having already included lags of $y$. i.e. Does including additional variables give better predictability?

Granger causality tests test predictability/correlation, not causation.

Note it is near impossible to detect causal relationships from just visually inspecting the series.

In addition, a scatter plot can show the direction of association, but not identify the direction of causality.

Same as correlation matrix.

### 2. Types of Granger Causality


1.   Biivariate causality (variable directly GC another variable)


> a. Unidirectional fail to GC

> b. Unidirectional causality

> c. Bidirectional causality (or feedback)

> d. Independence

2.   Multivariate causality



> a. X GC Y which then GC Z


> b. X GC Y and Z but Y and Z are independent (drawn as a tree), note a bivariate test between Y and Z would mistakenly identify a causal link between the two (spurious result), also if Y and Z fail to cause X, then X is considered exogenous i.e. not affected by the system, variables that are not exogenous are endogenous

### 3. Testing Granger Causality

Suppose you have a bivariate VAR with 2 lags.

If you want to test if Y2 granger causes Y1, you test the null hypothesis:

$H_0: \phi_{13} = \phi_{14}=0$ i.e. Y1 equation Y2 lags

$H_1:$ at least one restriction fails

We reject the null if the p-value is less than 0.05.


In addition, which optimal alg structure, if optimal lag structure is 0, this implies no causal linkage between the two variables.

### Week 6 - Multivariate Model Extensions

### 1. Impulse Response Analysis Motivation

Granger causality tests relative importance of variables in a VAR.

Granger causality doesn't tell us anything about the sign or direction of causality. It may only be obvious by looking at a 1 lag model and the sign of the coefficients. 

For more complicated models, this is more difficult. Hence, we compute impulse responses, which basically is feeding a 'shock' into the VAR and see how variables change over time.

### 2. Impulse Response Analysis - Cholesky Decomposition

With our regular VAR, we have have non-independent error terms from our equations. e.g. for a bivariate case, $E(u_{1t} u_{2t}) \ne 0$. i.e. The error covariance matrix $\Omega$ is non-diagonal.

This is a problem because a shock in $u_{1t}$ is difficult to interpret as this is not independent of a shock in the second equation. You want to see see how a variable changes due a shock, we can't see the causal link as it could also be responsible to another shock.

The aim is to obtain a strutural model where we have structural independent errors.

We want to re-write the model (more specifically the way to compute parameter coefficients) where your second equation contains the $y_{1t}$ in the second equation for $y_{2t}$ as an explanatory variable. 1st equation is the same.

The benefit is that we have new error terms $v_{1t}$ and $v_{2t}$ are independent i.e. $E(v_{1t} v_{2t}) = 0$.

This transformation of a VAR is known as:
1.   Cholesky decomposition
2.   Recursive structural VAR (SVAR)

e.g. If we had a trivariate case:
*   1st equation remains the same
*   2nd equation has the $y$ variable from the first equation
*   3rd equation has the $y$ variables from the first and second equations

Given this, the error terms are now independent. Off-diagonal elements of the new $\Omega$ is 0. Zero covariances, this means LS is asympottically efficient.

Given a shock in $v_{1t}$ of size $\sigma_1$ in the first period.

In the same first period:
*   $y_{1t}$ changes by $\sigma_1$
*   $y_{2t}$ changes by $b_4\sigma_1$
*   $y_{3t}$ changes by $c_4\sigma_1$

Given a shock in $v_{2t}$ of size $\sigma_2$ in the first period.

In the same first period:
*   $y_{1t}$ does not change as it has no $y_{2t}$ in the equation
*   $y_{2t}$ changes by $\sigma_2$
*   $y_{3t}$ changes by $c_5\sigma_2$

Given a shock in $v_{3t}$ of size $\sigma_3$ in the first period.

In the same first period:
*   $y_{1t}$ does not change as it has no $y_{3t}$ in the equation
*   $y_{2t}$ does not change
*   $y_{3t}$ changes by $\sigma_3$

To know what happens in the next period, rewrite the model for $y_{t+1}$ and see how our changes from period 1 affect it. You can also look at derivatives:
$\frac{\partial y_{1t+1}}{\partial v_{1t}} = a_1 \frac{\partial y_{1t}}{\partial v_{1t}} = a_1 \sigma_1$

Know how to do this with the 3rd equation.

We can estimate this system/SVAR by applyign least squares to each equation one at a time, given our new explanatory variables.

The size of the shocks $\Omega_1, \Omega_2, \Omega_3$ are the standard deviations of the error terms of each structural equation (standard errors of regression).

The computed SE of regression may require an adjustment, you need to make a degrees of freedom adjustment. 1st equation stays the same. But the second equation now has an additional explanatory variable and thus has 1 less degree of freedom. Hence, you make the adjustment:
$\hat \sigma_2 = 0.015900 \times \sqrt{(198 - 5)/(198 - 4))} = 0.015859$

3rd equation now has 6 variables, not 5. Change 5 to 6.

In Eviews, estimate the VAR, then click Impulse, 20 periods, no response standard errors, click OK. You can then plot the impulses. If you shock output for example, you can see the effect in different periods.





### 2. Variance Decomposition

From Impulse Response, we can learn about the dynamics and the direction of causality, but not what is important or not.

Variance decomposition allows us to do this, using the same information from the impulse response, figuring out the relative importance of shocks in the system.

Impulse responses are about the conditional mean, variance decomposition is about the conditional variance.

Consider the VAR. We obtain a SVAR, where the shocks $v_{1t}, v_{2t}, v_{3t}$ are independent.

We define the 1-step forecast error variance of $y_{1t}$ conditional on information at $t - 1$ as:

$e_{1t} = y_{1t} - E_{t-1}(y_{1t}) = v_{1t}$

The one-step ahead forecast error for $y_{1t}$ is simply equal its own structural shock $v_{1t}$ because $E_{t-1}(v_{1t}) = 0$.

The forecast error variance conditional on last periods information is:

$var_{t-1}(e_{1t})=E_{t-1}(e^2_{1t}) = E_{t-1}(v^2_{1t}) = \sigma^2_1$

Because the forecast error mean is 0 and the definition of the error variance being $\sigma^2_1$.

Hence, for the first equation, we can decompose the forecast variance due to its own shocks ($y_{1t}$) or by other shocks ($y_{2t}, y_{3t}$).

Total variance is: $var_{t-1}(e_{1t})$

Variance due to its own shock: $\sigma^2_1$

Hence, the proportion due to itself is 1 or 100%, and the proportion due to other shocks is 0 or 0%.

An estimate of the 1-step ahead forecast variance for $y_{1t}$ is: 

$\hat var_{t-1}(e_{1t})=\hat \sigma^2_1 = 0.011799^2$

Do the same for the forecast error variance of $y_{2t}$.

Based on its expectation, the error variances $v_{1t}$ and $v_{2t}$ are 0. Hence, the difference is:

$e_{2t} = y_{2t} - E_{t-1}(y_{2t}) = v_{2t} + b_4 v_{1t}$

i.e. The forecast error is due to both variable 1 shocks and variable 2 shocks.

The solve for the forecast error variance, we square the term $e_{2t} = y_{2t} - E_{t-1}(y_{2t}) = v_{2t} + b_4 v_{1t}$, expand it and obtain:

$E_{t-1}(v^2_{2t} + b^2_4 v^2_{1t} + 2v_{2t}v_{1t})$

Because we know that the error terms are independent and the definition of the error variances, the forecast error variance of $y_{2t}$ is:

$var_{t-1}(e_{2t}) = \sigma^2_2 + b^2_4 \sigma^2_1$

The proportion of forecast error variance 1-step ahead for $y_{2t}$ due to itself is $\frac{b^2_4 \sigma^2_1}{var_{t-1}(e_{2t})}$ and due to $y_{2t}$ shocks is $\frac{\sigma^2_2}{var_{t-1}(e_{2t})}$ and proportion due to $y_{3t}$ is 0.

We know all these variables, so we can compute these variances and the proportion of the variance due to itself and others.

Eview: VAR -> View / Variance Decomposition

You can look at short-run (1 year) and long-run (5 year ) effects, where you can check the significant causal patterns between variables (e.g. based on contributions above 10%).




###3. Diebold-Yilmaz Spillover Index

1 application of the variance decomposition.

Used to show how markets are connected, their interdependence (where forecase error variance comes from, its own market or other markets) and performs network analysis.

You pick a particular time horizon. e.g. if you are interested in a business cycle, maybe 5-year cycle.

We obtain the variance decomposition matrix.

The spillover index tells us the total variability that is due to spillovers between markets (i.e. not from itself). e.g. if you had 3 variables, and 90% of variability comes from itself, the idnex would be equal to 300% - 90%/300% = 210/300 = 70%. 

You build a VAR, based on the market returns, and choose the optimal lag based on AIC, SIC and HQ.

Then compute the forecast error and variance based on the difference between the realised value and the expected value based on the VAR.


### 4. Nonrecursive SVARs

Sims critical of recursive (changing the ordering of the variables) macroeconomic models (variable 1 feeds into variable 2, variable 2 feeds into variable 3), because it is restrictibe in terms of economic theory.

Cheolesky decomposition has the same criticism.

The end of the day, ordering is not justified by economic theory.

Hence, non-recursive structure seems to make more economic sense.

Models can be estimated using Eviews, but outside of the course (estimation more involved).



### Week 7 - Nonstationary Models: Unit Roots

VAR variables are actually non-stationary.

The question: can you estimate a model that is non-stationary? Depends on co-integration.

We first test formally for non-stationary.

### 1. Random Walks vs. Trend Stationarity

Two different models of non-stationary.

We typically have variable non-stationarity of a random-walk type.

We have previously looked at stationary in the mean or variance or higher moments such as skewness.


2 broad methods for generating non-stationarity series:
1.   Deterministic time trend (adds linear line going up $y_t = \alpha + \beta TREND_t + u_t$)
2.   Random walk without drift ($y_t = y_{t-1} + u_t$) and with drift ($y_t = \phi_0 + y_{t-1} + u_t$), stationary in the mean (without drift), non-stationary in the variance, non-stationary in the mean (with drift)

E.g. With AR(1) model, if $\phi_1 = 1$, then the model appears to be a random walk with drift. Models thus are nonstationary if this is very close to 1.

We can generate models and get numbers close to 1. Is it due to standard error or is it statistically smaller than 1? We need a formal test.

Nelson-Plosser said that almost all macro variables have the form of a random walk, rather than have a deterministic trend - this changes the way we analyse variables.

What is the implication if we have a random walk variable?


*   Forecasting: when you take conditional expectations and use OLS/MLE estimates for random walk with drift, we get $\hat y_{t+h} = y_t + H\hat \phi_0$, forecast simply follows a linear time trend, without drift, you prediction is always $y_t$, which effectively tells you efficient market hypothesis - all public information is included in the current price 

Integradedness: How many times you first difference a variable to achieve stationarity
*   I(1) non-stationary and random walk, differencing once to achieve stationarity
*   I(0) already stationary
*   I(2) differencing twice to achieve stationary, graphically more smoothly evolving, e.g. nominal economic series

Random walk requires differencing once. 

There is no evidence that any economic series are I(3) or higher.






### 2. Unit Root Tests and Testing Different Order Nonstationarity

We want to test for an AR(1) model if there is a unit root/nonstationarity.

Null: = 1 i.e. I(1) / nonstationary
Alternative: < 1 i.e. I(0) stationary

Test by:
*   Estimate AR(1) model
*   Conduct t-test ($tstat = \frac{\hat \phi_1 - 1}{se(\hat \phi_1)}$)
*   However: this statistic does not follow a student-t distribution or normal distribution under the null hypothesis, the tables you choose depend on the null hypothesis, which is nonstationary, distribution of the D-F statistic is non-standard, the correct distribution is the D-F distribution, negatively skewed
*   Look up the right p-value under the D-F distribution

D-F Test:
*   Start with AR(1) model
*   Subtract $y_{t-1}$ from both sides, now the coefficient given unit root is 0
*   This means we can now test for statistical significance and use the right p-value because Eviews automatically choosing the unit root test statistic and p-value
*   Standard errors of the transformed coefficients are the same because we only did a linear transformation $se(\hat \phi_1) = se(\hat \beta)$
*   Null is beta = 0 and beta being less than 0.
*   Note: standard errors don't change, nor tstat value, p-value is incorrect (pick the right distribution for the t-statistic)
*   In addition, D-F test assumes errors are white noise, but we fit a AR(1) model, without testing lags and fit, unlikely to meet this assumption

Augmented D-F Test:
*   Start with the VAR, find optimal p based on AIC, etc.
*   Subtract $y_{t-1}$ from both sides
*   You again test beta = 0 but you now have all the $p$ $y_{t-1} - y_{t-i-1}$ terms
*   This aims to soak up autocorrelation from the residuals and get closer to white noise
*   Essentially the difference is that we choose the number of lags for our model in which we test, and Eviews chooses this p based on the information criteria
*   Null: Non-stationarity? Doe that imply I(1)? Could also be higher order non-stationarity
*   You do that test again for the first-differenced series to see if its I(1) or I(2)














### 3. Other Unit Root Tests

There are different unit root tests - the difference is the 'power' of a test - the probability of choosing the alternative when it is true:
*   Extending to allow for time trend (resulting in different critical values)
*   Allowing for structural breaks (resulting in different critical values)
*   Phillips-Perron (nonparametrric autocorrelation adjustment)
*   KPSS (null hypothesis is stationary)
*   Elliott-Rothenberg-Stock (improvement in power)

Interesting extension: price bubbles, where null is unit root, but alternative is phi > 1, indicatign explosive unit root or price bubble. Don't do whole sample, do rolling windows or expanding windows.



### Week 8 - Nonstationary Models: Cointegration

Equilibrium talk and convergence doesn't seem to make sense if there is non-stationarity.

It is possible, you don't talk about a static equillibrium (feed a shock, variable converges to a single point), we talk about dynamic long-run equilibrium (that changes all the time, but processes are attracted to each other).

### 1. Long-Run Economic Behaviour

### 2. Cointegration Definition

We have 2 variables that must be non-stationary.

Visually, you can do a scatter diagram of 2 variables and see a linear line.

Consider you model a linear relationship between these 2 variables:

$y_{1t} = \beta_0 + \beta_1 y_{2t} + e_t$

If deviations do not persist for too long i.e. $e_t$ is I(0), then the movement of 1 variable is determined by the movement of another variable. This doesn't mean its white noise, it can exhibit autocorrelation.

Cointegration is if the 2 variables are I(1), you model them in a linear relationship, and the difference/residual is stationary I(0), then the 2 variables are cointegrated.

Key implication is that this equation is a long-run equation because the deviations are temporary, thus we move towards to this relationship.

More generally, if 2 variables are I(2) and the errors in the linear relationship is lower, even I(1), it is cointegrated.

### 3. Spurious Correlation

Spurious correlation is where two variables don't have any economic connecting process, but they are found to be highly (positively or negatively) correlated.

This can arise when 2 variables are non-stationary but are not cointegrated. 

You will find that the higher degree of non-stationarity, there tends to be higher correlation, generating high R-squareds when we model it, which is purely spurious!



### 3. Cointegration Equation Estimation

We start with a linear equation.

We have different co-integrating estimators:
*   Engle-Granger estimator i.e. LS (single equation)
*   Johansen estimator (multiple equation)

These estimators have an important statistical property:
*   Because our variables are non-stationary and co-integrated,  we have super/T consistency where it converges to the population beta at rate T
*   Our standard estimators, which assume stationarity, are consistent at the usual rate of root T

Engle-Granger estimator i.e. LS (single equation)
*   Regress using OLS
*   Look at residuals to see if stationary, they may stil have autocorrelation, they are zero mean due to including a intercept



### 4. Cointegration Testing


*   Estimate model using Engle-Granger / OLS.
*   Then compute OLS residuals.
*   Then apply ADF unit root test on residuals.

However,the residuals is based on our estimated model, and we need to recognise the loss in the degrees of freedom used in the estimation of our model and thus our residuals, thus we need to generate the correct critical values and p-values.

The Johansen cointegration test:
*   Multivariate generalisation
*   First stage: null no cointegration for N-variate system both variables only non-stationary, alternative: at least one cointegrating equation
*   2nd stage: null one cointegratign equation, alternative at least two cointegrating equations
*   Stage N: null N - 1 cointegrating equations, alternative: all variables stationary I(0)

When doing this test, it gives us a eigenvalue, a trace statistic, the 0.05 critical value and the p-value.




### Examples of Well Known Cointegrating Relationships



*   Term structure of interest rates
*   Permanent income hypothesis (real consumption, real GDP)
*   Money demand (real GDP, real money, interest rates)
*   Fisher equation (interest rate, inflation)
*   Present value model (stock price, dividends)
*   Purchasing power parity (exchange rate, domestic and foreign prices)





### Week 9 - Nonstationary Models: VECM

### 1. Motivation



*   We did unit root tests to test if variables are I(1) or I(2)
*   We did cointegration tests to see if there is long-run cointegrating relationships i.e. error is I(0), etc.
*   Now we aim to estimate this cointegrating equation, called a Vector Error-Correction Model or VECM



### 2. Vector Error-Correction Model 



*   Given a shock in the error term, there are 3 possible dynamics paths: variable y1 adjusts, variable y2 adjusts, or both adjust such that the long-term relationship holds
*   The size of the disequilibrium is thus the error term
*   The error correction model tells us that the change of the variable / theri dynamic time paths are determined by the size of the equilibrium and a 'speed of adjustment' variable

$e_t = y_{1t} - \beta_0 + \beta_1 y_{2t}$

$\Delta y_{1t+1} = \alpha_1 + \gamma_1 e_t + v_{1t+1}$

$\Delta y_{2t+1} = \alpha_2 + \gamma_2 e_t + v_{2t+1}$

The gamma parameters represent the 'error-correction' parameters. For example if $\beta_1 > 0$ (positively sloped long-run equation), if $\gamma < 0$, then $y_{1t}$ adjusts downwards.




### 3. Estimation: Single Equation Methods

We need to estimate the following parameters:

$\theta = [\beta_0,\beta_1,\alpha_1,\alpha_2,\gamma_1,\gamma_2]$

i.e. The long-run relationship, the speed of adjustment and the direction of the change.

Single equation methods are based on OLS applied to each equation one at a time.
*   Advantage: parameter estimates are consistent (super-consistent in the case of $\beta_1$ i.e. at rate T, rest at rate square root T)
*   Disadvantage: not asympotically efficent, because estimators not computed jointly

Steps:
1.   Regress $y_{1t}$ on $c, y_{2t}$ to obtain cointegrating parameters $\beta_0, \beta_1$ using OLS
2.   Obtain time series of residuals $\hat e_{t-1}$
3.   Regress $\Delta y_{1t}$ on $c, \hat e_{t-1}$ to obtain first equation parameters $\alpha_1, \gamma_1$ using OLS
3.   Regress $\Delta y_{2t}$ on $c, \hat e_{t-1}$ to obtain second equation parameters $\alpha_2, \gamma_2$ using OLS


### 4. Estimation: Systems Equation Methods

To achieve asymptotic efficiency, all parameters are estimated jointly using a MLE known as the Johansen estimator.
*   Can perform hypothesis tests on VECM parameters based on asymptotic normality

Implementing the Johansen estimator:
*   Specify error distribution of N-variance VECM, which we usually assume as multivariate normality, $v_t = (v_{1t},v_{2t},...,v_{Nt})' -> N(0,\Omega)$
*   $\Omega = E(v_t v_t')$ is the covariance matrix of the VECM errors
*   The average log-likelihood of the multivariate normal distribution is: $\frac{\log L(\theta)}{T} = -\frac{N}{2} - \frac{1}{2}\log |\Omega|-\frac{1}{2T}\sum^T_{t=2} v_t' \Omega^{-1} v_t$
*   For a model, use the first VECM equation and substitute the error term from the long-run cointegrated equation, so we have only y variable terms, subtituting out errors
*   We need to estimate the following parameters: $\theta = [\beta_0,\beta_1,\gamma_1,\gamma_2, \sigma^2_1, \sigma^2_2, \sigma_{12}]$
*   Choose the unknown parameters to maximise the average log-likelihood

In Eviews:
*   Open VAR ... / Vector Error Correction
*   Change lag structure to 0 lags , 0 0 
*   We need more lag veriables to ensure VECM errors are white noise, which is required as assumption
*   How we choose the lag structure, we obtain the restricted VAR, where if VAR lag structure is p, VECM lag structure is p - 1, hence we can determine this using information criteria for a lag length p for a VAR where the variables are expressed in levels provided the variables are cointegrated (if not, variables should be expressed in first differences)




### 5. Forecasting

### Week 10 - Volatility Models GARCH

### 1. Motivation

Switch to the mean to the variance.

Useful in risk management, VaR, option pricing, long-run marginal expected short-fall, predictive regressions, interaction between volatility and the real economy.

Models of volatility:
*   Generalised Autoregressive Conditional Heteroskedasticity (GARCH)
*   Realised Volatility/Variance (RV)
*   Stochastic Volatility (SV)
*   Implicit Volatility (IV)
*   Range Volatility

We see with return series tends to have bursts of volatility with positive autocorrelation in the variance, with quiet and more volatile periods, almost like 2 regimes. You are now saying the variance is non-constant over time, hence heteroskedastic.

Descriptive statistics are based under the assumption of a constant variance, which suggests misspecification.

Autocorrelation of returns shows no significant AR relationship, based on the ACF and PACF, i.e. no predictability.

However, we see autocorrelation in squared returns, i.e. predictability.
Note: this is the variance of returns provided the mean return is zero. If it is ver small, we could treat squared returns as a reasonable approximation.


### 2. Testing for Constant or Time-Varying Variance

Obtain squared-demeaned return series i.e. variance.

Do a AR equation and check the ACF/PACF.

Test the following hypothesesis:
*   Null all coefficients for AR terms are jointly 0 (constant variance)
*   At least one restriction is violated (time-varying variance)

This test represents a preliminary test for ARCH and denoted as ARCH(q).

Returns are represented as:
$r_t = \phi_0 + u_t$

Squared demeaned returns is the variance of the series:
$u_t^2 = (r_t - \phi_0)^2$

In practice, we would estimate the above return model and extract the model residuals $\hat u_t$.

Then you estimate a ARCH(q) or a AR(q) model for the variance:

$\hat u_t^2 = \delta_0 + \delta_1 \hat u_{t-1}^2 + ... + \delta_q \hat u_{t-q}^2 + v_t$

Then you compute the test statistic: 

$ARCH = TR^2$ where T is the sample size and $R^2$ is the coefficient of determination from the regression.

Under the null hypothesis of constant variance, the test statistic is distributed under a chi-squared distribution with q degrees of freedom. We can estract the appropriate critical values and p-values from this distribution.

Given this, if the test statistic p-value is greater than 0.05, we fail to reject the null at the 5% level. If it is less than 0.05, we reject the null in favour of the alternative. This procedure is formally known as the Lagrange multiplier test.

Eviews: Quick / Estimate Equation / r c / View / Residual Diagnostics / Heteroskedasticity Tests / ARCH / q / Ok.





### 3. Autoregressive Conditional Heteroskedasticity (ARCH) Model Specification

ARCH models capture autocorrelation in the variance. 

The variance is determined by the size of previous shocks.

e.g. the ARCH(1) model is:

$r_t = \phi_0 + u_t$

$h_t = \alpha_0 + \alpha_1 u_{t-1}^2$

$u_t -> N(0,h_t)$

THe key features is that the conditional mean is a constant, and the distirbution of $u_t$ is normally distributed.

$f(r_t|r_{t-1},r_{t-2},..,\theta) = \frac{1}{\sqrt{2\pi h_t}} \exp(-\frac{(r_t - \phi_0)^2}{2 h_t})$

Interpretation is that small shocks $u_{t-1}$ will result in a small variance $h_t$ and vice-versa. Large shocks will result in an increase in variance in the next period if $\alpha_1 > 0$.

The "News Impact Curve" is a plot of $h_t$ against $u_t$. We see shocks of the same magnitude have the same effect on $h_t$ (no asymmetry effects).

A very specifal case of the ARCH(1) model is a constant variance $\alpha_1 = 0$. 

The model can be generalised to have more lags.

Estimating the ARCH model parameters:

$\theta = [\phi_0, \alpha_0, \alpha_1, .., \alpha_q]$

We estimate using maximum likelihood using an iterative alogorithm. Given conditional normality, the log-likelihood function is:

$\log L(\theta) = \frac{1}{T} \sum^T_{t=1} \log (r_t|r_{t-1},r_{t-2},...,\theta)$

$-\frac{1}{2} \log (2\pi) - \frac{1}{2T}\sum^T_{t=1} \log h_t - \frac{1}{2T} \sum^T_{t=1} \frac{u^2_t}{h_t}$

Where $u_t = y_t - \phi_0$

$h_t = \alpha_0 + \sum^q_{i = 1} \alpha_i u^2_{t-1}$

Eviews: Quick / Estimate Equation / r c / Estimation settings: ARCH / ARCH order 1 / GARCH order 0 / OK

View / GARCH Graph / Conditional Variance/ Proc / Make GARCH Variance Series / OK






### 4. Is the Model a Good Fit? Diagnostics

We have estimated the ARCH model.

Is the model any good?

If the model is specified correctly, then the standardised reisiduals 

$z_t = \frac{u_t}{\sqrt{h_t}}$, the variance of it should be 1, i.e. $var(z_t) = 1$ and that there should be no evidence of autocorrelation in the variance. 

This is because our model assumptions is that the error term is normally distributed with 0 mean and variance $h_t$

Also, we want to check if we have the true process. E.g. What if the true process is an ARCH(2) when we have an ARCH(1)?

$Var(z_t) = \frac{var(u_t)}{h_t}$

If the correct model specification is done, that the true $var(u_t)$ and $var(h_t)$ are the same.

However, if for example the $var(u_t)$ is ARCH(3) and $h_t$ is ARCH(1), then the ratio would be ARCH(2) as the variance expressions in the numerator and denominator would not cancel out.

$u_t$ is the true variance and $h_t$ is the specified variance from the model.

Hence, we want to test the following hypothesis:
*   Null: model is correctly specified where $z_t$ variance is constant
*   Alternative: model is misspecified and $z_t$ is time-varying

i.e. We should see white-noise standardised residuals.

Eviews: View / Residual Diagnostics / ARCH LM Test / Number of Lags: 1 (is there another lag in the true model?)

We would get the ARCH test statistic and see its relevant p-value based on the chi-square distribution under trhe null. If less than 0.05, we reject the null of correct specification and conclude that the ARCH variance specification has higher order dynamics than ARCH(1) for example.

We keep estimating a new model, reapplying the test until we can no longer reject the null.


### 5. Generalised ARCH (GARCH)

When estimating ARCH models, you find that you generally need a fairly long q. i.e. Volatility seems to have long-memory and is persistent. 

This can mean using a lot of degrees of freedom and lags.

Like MA, we can capture this persistence where variance tends to cluster in periods while controlling for the number of parameters, we add an additional explanatory variable in the lagged conditional variance. 

So the GARCH term is responsible for capturing volatility persistence and memory features of volatiliy, which means that many lagged volatilities has predictability on current and future volatility. The GARCH terms allow for an infinite ARCH structure. Same ARMA for volatility.

$h_t = \alpha_0 + \alpha_1 u^2_{t-1} + \beta_1 h_{t-1}$

We can show GARCH(1,1) to be a infinite ARCH model with just this additional parameter $\beta_1$.

We have the GARCH(1,1) model:
$(1-\beta_1 L)h_t = \alpha_0 + \alpha_1 u^2_{t-1}$

Assuming $|\beta_1|<1$, then we can invert the term:
$h_t = (1-\beta_1L)^{-1} \alpha_0 + (1-\beta_1L)^{-1} u^2_{t-1}$

$= \frac{\alpha_0}{1-\beta_1}+\alpha_1(1+\beta_1 L + \beta_1^2 L^2 + ...)u^2_{t-1}$

$= \frac{\alpha_0}{1-\beta_1}+(\alpha_1 u^2_{t-1} + \alpha_1 \beta_1 u^2_{t-2} + \alpha_1 \beta^2_1 u^2_{t-3} + ...)$

Hence, we have an infinite order ARCH model.

The $\beta_1$ effectively tells us the effect of past shocks. The bigger the number, the bigger the strength of previous periods and thus, the longer the memory.

We can allow for q lags of the ARCH term and p lags of the GARCH terms, resulting in GARCH(p,q) model.

Again model assumptions have normal error term and conditional mean being constant.

The GARCH(1,1) model generally is the best model. The estimates of $\beta_1$ is around 0.9 and the $\alpha_1$ term is around 0.05.

Special case is ARCH(1) model is the constant variance model.

Given the assumption of normality, we can estimate the GARCH model parameters:

$\theta = [\phi_0, \alpha_0, \alpha_1, ..., \alpha_q,\beta_1,\beta_2,...,\beta_p]$

Using MLE. The log-likelihood is:

$\log L(\theta) = \frac{1}{T}\sum^T_{t=1} \log f(r_t|r_{t-1},r_{t-2},...;\theta)$

$=1\frac{1}{2}\log 2\pi - \frac{1}{2T}\sum^T_{t=1} \log h_t - \frac{1}{2T} \sum^T_{t=1} \frac{u^2_t}{h_t}$

$u_t = r_t-\phi_0$

$h_t = \alpha_0 + \sum^q_{i=1}\alpha_i u^2_{t-1} + \sum^p_{i=1} \beta_i h_{t-i}$

